In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

ls: cannot access '/home/kesci/input/': No such file or directory


In [12]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

lost+found  negmas_scml


# Advanced Negotiation 
## Modeling

Implemente the model in Negmas 

Version -- 21.Jul.2020

transfer the rlboa framework into the package negmas/modeling

filesystem structure

\negmas
	\modeling
		\rlboa
			\boaframework
			abstract_state.py
			state.py
			rlboa.py
			rlboa_agent.py
			qlearner.py
		acceptance_modeling.py
		utility.py
		strategy.py
		future.py



In [1]:
!pip install gym -i https://pypi.tuna.tsinghua.edu.cn/simple

    100% |████████████████████████████████| 1.6MB 853kB/s eta 0:00:01
    100% |████████████████████████████████| 1.0MB 1.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for gym ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/e2/f5/f5/3791255957de1b1e9b33a24419443c70334501bb1001b0372f
Successfully built gym
You are using pip version 9.0.1, however version 20.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
'''
[]
obsver_space = spaces.Dict(
{"X_best": None, "T_left": None, 
"IP_my": None, "RP_my": None, "S_neg": None})
'''

import gym
from gym.spaces import Box
help(gym.spaces.Box)
# k = Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32)

Help on class Box in module gym.spaces.box:

class Box(gym.spaces.space.Space)
 |  A (possibly unbounded) box in R^n. Specifically, a Box represents the
 |  Cartesian product of n closed intervals. Each interval has the form of one
 |  of [a, b], (-oo, b], [a, oo), or (-oo, oo).
 |  
 |  There are two common use cases:
 |  
 |  * Identical bound for each dimension::
 |      >>> Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32)
 |      Box(3, 4)
 |  
 |  * Independent bound for each dimension::
 |      >>> Box(low=np.array([-1.0, -2.0]), high=np.array([2.0, 4.0]), dtype=np.float32)
 |      Box(2,)
 |  
 |  Method resolution order:
 |      Box
 |      gym.spaces.space.Space
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, low, high, shape=None, dtype=<class 'numpy.float32'>)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 

In [33]:
'''
For example Issue ={'laptop_price'}

IP_b [300 − 350]
RP_b [500 − 550]
IP_s 100%[500 − 550], 60%[580 − 630], 10%[680 − 730]
RP_s 100%[300 − 350], 60%[380 − 430], 10%[480 − 530]
MD_H{30, 40, 50}, A{18, 23, 28}, L{8, 10, 12}
MR_H{10:1, 1:1, 1:10}, A{5:1, 1:1, 1:5}, L{2:1, 1:1, 1:2}
t_end Lg[151s –210s], A[91s –150s], Sh[30s –90s]
ZoA H(100%), A(60%), L(10%)

obsver_space = spaces.Dict(
{"X_best": None, "T_left": None, 
"IP_my": None, "RP_my": None, "S_neg": None})

X_best: the minimum offer offered by opponent.

A possible observation space:
[[300$ - 550$], [151s - 210s], [300$ - 350$], [500$ - 550$], [0 - 5]]

Example
[400, 160, 330, 530, 2]
First we just consider a simple negotiation scenario, in other words, a simple negotiation settings
Issue = {'laptop_price'}
IP_b = [300 - 350]
RP_b = [500 - 550]
IP_s = [500 - 550]
RP_b = [300 - 350]
MD   = [2]
MR   = [1:1]
t_end = [151s - 210s]
ZoA = H(100%)
as considered by paper 'A Deep Reinforcement Learning Approach to Concurrent Bilateral Negotiation'

a possible observation state is [350$, 20s, 340$, 530$, 2]
a possible action is a new offer outcome 
'''

import numpy as np
from gym import spaces
observation_space = Box(low=np.array([300, 151, 300, 500]), high=np.array([550, 210, 350, 550]), dtype=np.float32)

observation_space.sample()

array([401.2926 , 191.68828, 327.87628, 525.7085 ], dtype=float32)

In [35]:
'''
convert the the action defined by negmas.Action to action space that obtained in Gym.spaces
        actions that agent can select are [Offer(x), ResponseType.REJECT_OFFER, ResponseType.ACCEPT_OFFER, exit]
        ResponseType.ACCEPT_OFFER
        ResponseType.REJECT_OFFER, will be gived a chance to propose new offer. 
        ResponseType.END_NEGOTIATION
        ResponseType.NO_RESPONSE, refuse to offer.
        ResponseType.WAIT
        
        when call function respond_ and firstly get the action ResponseType.REJECT_OFFER, then will call the function proposal_ to 
        get the new offer, the final action in AnegmaEnv is Offer(x)
        
        the logic has been described in the paper `A Deep Reinforcement learning approach to concurrent negotiation`
        first condition: if just consider the propose, the action space is also a one dimensional box space for one issue.
        second condition: if consider the response, the action space is [0, 1, 2, 3, 4]
        
        0: Offer(x): the outcome, if the response is ResponseType.REJECT_OFFER, the info sent to opponent is a offer
        1: ResponseType.ACCEPT_OFFER
        2: ResponseType.END_NEGOTIATION
        3: ResponseType.NO_RESPONSE
        4: WAIT
        how to define the action_space
        
        first we consider the first condition, the action is the propose
'''

action_space = spaces.Box(low=np.array([300]), high=np.array([550]), dtype=np.float32) 

In [40]:
'''
Reward: 
    first condition: just consider the regression problem, means counter-offer, create a new offer
    second confition: classfication problem, ResponseType.Accept, get a agreement, 
    ResponseType.No_RESPONSE or ResponseType.END_NEGOTIATION, other WAIT

for first condition, just calcualte the utility of new offer

'''
t_end = 500.0
current_t = 200.0
d_t = 0.5
state =  observation_space.sample()
x:"new offer by my negotiator" = 340
reward = ((state[3] - x) / (state[3] - state[2])) * (current_t / t_end)**d_t
reward = 0 # if the current_t greater than t_end
reward = -1 # if the new offer is greater than  offer that created by opponent

0.5678369673327723